In [46]:
#write script to find all of the files that we have alread run caiman on and load the object, recalculat the dff_f with the corrrect window, and then resave the hd5
#and the mat file so that the mat file also has cleaner transients. 
import os 
import glob
from caiman.source_extraction.cnmf.utilities import detrend_df_f
from caiman.source_extraction.cnmf.cnmf import load_CNMF
import time as t
import datetime as dt
from scipy.io import savemat
import numpy as np
from caiman.utils.visualization import get_contours
import caiman as cm

t1 = dt.datetime.now()
timeStamp = str(t1.year)  + '_' + str(t1.month) + '_' + str(t1.day) + '_' + str(t1.hour) + '_' + str(t1.minute)
print(timeStamp)
rootDirs = ['/mnt/Brenna_Feb2023']
pattern = 'caimanEval_Python.hdf5'
bigList = []
matList = []
for cDir in range(len(rootDirs)):
    rootDir = rootDirs[cDir]

    folderList = []
    for path in glob.glob(f'{(rootDir)}/*/**/', recursive=True):
        folderList.append(path)

    # folderList = [folder for folder in files if os.path.isdir(os.path.join(rootDir, folder)) == True]
    for cFol in range(len(folderList)):
        filesIn = os.listdir(os.path.join(rootDir, folderList[cFol]))

        for cF in range(len(filesIn)):
            currentPath = os.path.join(rootDir, folderList[cFol], filesIn[cF])
            if pattern.lower() in filesIn[cF].lower():
                bigList.append(os.path.join(rootDir, folderList[cFol], filesIn[cF]))



2023_4_7_14_7


In [60]:
def saveCaimain2MatDifferent(estimates, saveLocation, outName, dims):
  from caiman.utils.visualization import get_contours
  fields = vars(estimates)
  matInfo ={}
  #dictionary of vars that we are actually interested in taking from the CNMF object 
  varsWeWant = {
          'C':'temporal_components', 
          'b':'bckgrnd_spatial', 
          'f':'temporal_bckgrnd_components', 
          'YrA':'residual_components', 
          'F_dff':'dFF',
          # 'idx_components':'good_components',
          # 'idx_components_bad':'bad_components',
          'SNR_comp':'SnR',
          'r_values':'rVals',
          'cnn_preds':'cnn'
          }
  
  #making the sparse matrix into a 3D matrix that matlab will actually be able to parse
  numRois = np.shape(estimates.A)[1]
  spatialComponents = np.zeros([dims[0], dims[1], numRois])
  for cROI in range(np.shape(estimates.A)[1]):
    #changed cROI - 1 to cROI
    roiArray = np.reshape(estimates.A[:,cROI].toarray(), dims, order='F')
    spatialComponents[:,:,cROI] = roiArray

  #using an internal caiman function to extract the contours of our ROIs
  caimanContours = get_contours(estimates.A, dims)

  #manually add a couple of variabeles to our matInfo dictionary
  matInfo['ica_segments'] = spatialComponents 
  matInfo['ROIContours'] = caimanContours

  #loop through the vars we want and save the corresponding variables into our matInfo dictionary
  for cK in varsWeWant.keys():
    matInfo[varsWeWant[cK]] = fields[cK]
  #use the scipy.io function to save out a .mat file from a dicionary input variable
  savemat(os.path.join(saveLocation, outName + '.mat'), matInfo)

  return matInfo

In [61]:
for cS in bigList:
    caimanSession = load_CNMF(cS)

    currentFolder = os.path.dirname(cS)
    currentFile = os.path.basename(cS)
    estimates = caimanSession.estimates
    print("Working on: ", os.path.join(currentFolder, currentFile))
    print("calculating df_f...")
    estimates.F_dff = detrend_df_f(estimates.A, estimates.b, estimates.C, estimates.f, YrA=estimates.YrA, quantileMin = 8, frames_window = 500)

    print("Saving new files...")
    dictVersionOfMat = saveCaimain2MatDifferent(caimanSession.estimates,currentFolder, currentFile + '_caimanEval_newDFF_calc' + timeStamp, caimanSession.dims)
    caimanSession.save(os.path.join(currentFolder, currentFile + '_caimanEval_Python_newDFF_calc.hdf5'))

Working on:  /mnt/Brenna_Feb2023/TdTG7_18/TdTG7_18_007/field1_gui/caimanOuputs_2023_3_15_1_20/ConcatenatedMovie_MC_reg_caimanEval_Python.hdf5
calculating df_f...
Saving new files...
Working on:  /mnt/Brenna_Feb2023/TdTG7_18/TdTG7_18_007/field1_track/caimanOuputs_2023_3_15_1_35/ConcatenatedMovie_MC_reg_caimanEval_Python.hdf5
calculating df_f...
Saving new files...
Working on:  /mnt/Brenna_Feb2023/TdTG7_18/TdTG7_18_007/field2_track/caimanOuputs_2023_3_15_3_26/ConcatenatedMovie_MC_reg_caimanEval_Python.hdf5
calculating df_f...
Saving new files...
Working on:  /mnt/Brenna_Feb2023/TdTG7_18/TdTG7_18_008/field1_gui/caimanOuputs_2023_3_14_22_22/ConcatenatedMovie_MC_reg_caimanEval_Python.hdf5
calculating df_f...
Saving new files...
Working on:  /mnt/Brenna_Feb2023/TdTG7_18/TdTG7_18_008/field1_gui/caimanOuputs_2023_3_17_14_59/ConcatenatedMovie_MC_reg_caimanEval_Python.hdf5
calculating df_f...
Saving new files...
Working on:  /mnt/Brenna_Feb2023/TdTG7_18/TdTG7_18_008/field1_gui/caimanOuputs_2023_